In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import defaultdict
from random import sample

In [3]:
class DecisionNode():
    '''
    Constructor for decision node
    attr - attribute to be tested on
    thresh - threshold for numerical attr, if categorical set to none
    children - dict mapping attr value to child, if numeric child keys are 'le' and 'gr'
    label - class label. default None, if not none, indicates that this is a leaf
    '''
    def __init__(self, attr, children, label=None, thresh=None):
        self.attr = attr
        self.children = children
        self.label = label
        self.thresh = thresh

def learn_tree(data,attr_list,do_ig,attr_vals,targets,min_size_split=0,min_gain=0.0,max_depth=-1):
    '''
    Function to create decision tree based on data
    do_ig - boolean, True to use info gain, False to use gini critereon
    attr_vals - dict, maps to array of possible values for each attr, empty if numeric
    targets - array, possible values for target class
    set_depth - float, to set as the proportions of instances to create a leaf node
    min_size_split - int, min number of instances required in data before returning a leaf node
    min_gain - float, min info gain to determine when to stop
    max_depth - int, if -1 no depth limit, else maximum depth before stopping 
    '''
    #if num data too small, return leaf
    if len(data) < min_size_split:
        return DecisionNode(None,None,data['class'].unique()[0],None)
    #if max depth reached, return leaf
    if max_depth == 0:
        return DecisionNode(None,None,data['class'].mode()[0],None)

    #find best attr from list using info gain
    max_gain = float('-inf')
    min_gini = float('inf')
    best_attr = ''

    random_attr = sample(attr_list,int(math.sqrt(len(attr_list))))

    for attr in random_attr:
        if do_ig:
            ig_attr,thresh = get_info_gain(data,attr,len(attr_vals),targets)
            if ig_attr > max_gain:
                max_gain = ig_attr
                best_attr = attr
                best_thresh = thresh
        else:
            gini_attr = get_critereon(data,attr)
            if gini_attr < min_gini:
                min_gini = gini_attr
                best_attr = attr

    #if info gainf rom split not sufficient, return leaf
    if max_gain < min_gain:
        return DecisionNode(None,None,data['class'].mode()[0],None)

    best_vals = attr_vals[best_attr]
    best_children = defaultdict(lambda: DecisionNode(None,None,data['class'].mode()[0],None))
    
    #if numeric attr
    if len(best_vals) == 0:
        best_children['le'] = learn_tree(data.loc[data[best_attr] <= best_thresh],attr_list,do_ig,attr_vals,targets,min_size_split,max_gain,max_depth-1) if len(data.loc[data[best_attr] <= best_thresh]) > 0 else DecisionNode(None,None,data['class'].mode()[0],None)
        best_children['gr'] = learn_tree(data.loc[data[best_attr] > best_thresh],attr_list,do_ig,attr_vals,targets,min_size_split,max_gain,max_depth-1) if len(data.loc[data[best_attr] > best_thresh]) > 0 else DecisionNode(None,None,data['class'].mode()[0],None)
    else:
        for i in best_vals:
            best_children[i] = learn_tree(data.loc[data[best_attr] == i],attr_list,do_ig,attr_vals,targets,min_size_split,max_gain,max_depth-1) if len(data.loc[data[best_attr] == i]) > 0 else DecisionNode(None,None,data['class'].mode()[0],None)

    return DecisionNode(best_attr,best_children,thresh=best_thresh)


#Helper functions for decision tree
def get_info_gain(data,attr,num_cat,targets):
    split_entropy = 0
    if num_cat > 0:
        best_split = None
        for i in range(num_cat):
            split_data = data.loc[[attr] == i]
            split_entropy += (entropy(split_data,targets)*len(split_data))/len(data)
    else:
        sorted = data.sort_values(by=[attr])
        sorted['mean'] = (sorted[attr] + sorted[attr].shift(-1))/2
        sorted['entropy'] = sorted['mean'].apply(lambda row: split_thresh(sorted,attr,row,targets))
        best_row = sorted.nsmallest(1,['entropy'])
        best_split = best_row['mean']

    return (entropy(data,targets) - split_entropy),best_split

#determine split threshhold for numeric attr
def split_thresh(data,attr,thresh,targets):
    if math.isnan(thresh):
        return entropy(data,targets)
    df_le = data.loc[data[attr] <= thresh]
    df_g = data.loc[data[attr] > thresh]
    return (entropy(df_le,targets)*len(df_le) + entropy(df_g,targets)*len(df_g))/len(data)

def entropy(data,targets):
    if len(data) == 0:
        return 0
    ent = 0
    for i in range(len(targets)):
        try:
            prob_i = data['class'].value_counts().iloc[i]/len(data)
            ent -= (prob_i * math.log(prob_i,2))
        except:
            pass
    return ent

def get_critereon(data,attr):
    data_0 = data.loc[data[attr] == 0]
    data_1 = data.loc[data[attr] == 1]
    data_2 = data.loc[data[attr] == 2]
    return (gini(data_0)*len(data_0) + gini(data_1)*len(data_1) + gini(data_2)*len(data_2))/len(data)

def gini(data):
    if len(data) == 0:
        return 0
    prob_0 = data['class'].value_counts().iloc[0]/len(data)
    crit = prob_0 ** 2
    if len(data['class'].value_counts()) > 1:
        prob_1 = data['class'].value_counts().iloc[1]/len(data)
        crit += prob_1 ** 2
    else:
        pass
    return crit

In [ ]:
#train tree on training data
wine_df = pd.read_csv('datasets/hw3_wine.csv',delim_whitespace=True)
w_train,w_test = train_test_split(wine_df,test_size=0.2)
wine_attr = defaultdict(list)
wine_targets = [1,2,3]
decision_tree = learn_tree(w_train,list(w_train.columns.values).remove('class'),True,wine_attr,wine_targets)

#function to classify an instance based on decision tree
def classify(instance):
    node = decision_tree
    while node.label is None:
        if node.thresh is None:
            node = node.children[instance[node.attr]]
        else:
            node = node.children['le'] if instance[node.attr] <= node.thresh else node.children['gr']
    guess = node.label
    actual = instance['class']
    return guess

#test accuracy on training data
def test_decision(to_train,data):
    predictions = to_train.apply(lambda row: classify(row), axis=1)
    actual_labels = data.loc[predictions.index,['class']]
    return predictions.eq(actual_labels['class'].values).mean()

accuracy_test = []
accuracy_train = []



'''for i in range (100):
    voter_train, voter_test = train_test_split(voter_df,test_size=0.2)
    attributes = list(voter_train.columns.values)
    attributes.remove('target')
    decision_tree = learn_tree(voter_train,attributes,True)
    accuracy_train.append(test_decision(voter_train))
    accuracy_test.append(test_decision(voter_test))'''